#### Austin McOmber
#### A01948802
#### Fin 5350
#### HW 2

In [44]:
import numpy as np
from scipy.stats import binom


In [45]:
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)


In [46]:
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [47]:
#European Binomial Option Pricer
def euroBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
        
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            
    return Ct[0]
    


In [48]:
def eurBinomPricer(S, K, r, q, T, n, u, d, payoff, verbose = True):
    nodes = n + 1
    h = T/n
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
    Bt = np.zeros((nodes,n+1))
    
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
         
    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
            Bt[j, t] = Ct[j,t]-(Dt[j,t]*St[j,t])
            
    return Dt,Bt,Ct         

In [87]:
def americanBinomPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes=n+1
    h=T/n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
        
    for t in range((n-1), -1, -1):
        for j in range(t):
            Ct[j]=disc*(pu*Ct[j]+pd*Ct[j+1])
            St[j]=St[j]/u
            earlyPay=K-St[j]
            Ct[j]=np.maximum(Ct[j],earlyPay)
            
    return Ct[0]
            


PROBLEM 1

In [51]:
eurBinomPricer(100,105,0.08,0,0.5,1,1.3,0.8,callPayoff,verbose=False)

(array([[0.5, 0. ],
        [0. , 0. ]]), array([[-38.43157757,   0.        ],
        [  0.        ,   0.        ]]), array([[11.56842243, 25.        ],
        [ 0.        ,  0.        ]]))

Delta = 0.5; B = -38.42; Premium = 11.57


In [53]:
eurBinomPricer(100,105,0.08,0,0.5,1,1.3,0.8,putPayoff, verbose=False)

(array([[-0.5,  0. ],
        [ 0. ,  0. ]]), array([[62.45131354,  0.        ],
        [ 0.        ,  0.        ]]), array([[12.45131354,  0.        ],
        [ 0.        , 25.        ]]))

Delta = -0.5; B = 62.45; Premium = 12.45;

PROBLEM 2

In [57]:
eurBinomPricer(100, 95, 0.08, 0, 0.5, 1,1.3, 0.8, putPayoff, verbose=False)

(array([[-0.3,  0. ],
        [ 0. ,  0. ]]), array([[37.47078813,  0.        ],
        [ 0.        ,  0.        ]]), array([[ 7.47078813,  0.        ],
        [ 0.        , 15.        ]]))

European Put Price = 7.47


In [61]:
eurBinomPricer(100, 95, 0.08, 0, 0.5, 1,1.3, 0.8, callPayoff, verbose=False)

(array([[0.7, 0. ],
        [0. , 0. ]]), array([[-53.80420859,   0.        ],
        [  0.        ,   0.        ]]), array([[16.19579141, 35.        ],
        [ 0.        ,  0.        ]]))

European Call price = 16.196

The option is overvalued at 17 therefore the arbitrage opportunity consists of borrowing $53.804 and buying .7 shares.  Sell the call option, Arbitrage profit is .804

The option is undervalued at 15.5 therefore the arbitrage opportunity conisits of buying a call option and lending 53.804 and sell .7 shares.  Arbtrage profit = .696

PROBLEM 3


In [65]:
eurBinomPricer(100, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.69120742, 1.        , 0.        ],
        [0.        , 0.225     , 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-49.12704895, -91.27499672,   0.        ],
        [  0.        , -13.83536792,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[19.99369346, 38.72500328, 74.        ],
        [ 0.        ,  4.16463208,  9.        ],
        [ 0.        ,  0.        ,  0.        ]]))

Su has Delta = 1.0; B = -91.27; Premium = 38.73

Sd has Delta = .225; B = -13.84; Premium = 4.16

S0 has Delta = .69; B = -49.13; Premium = 19.99

PROBLEM 4

In [68]:
eurBinomPricer(80, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.46505058, 0.77307692, 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-28.59620401, -61.79797673,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[ 8.60784253, 18.60202327, 40.2       ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [69]:
eurBinomPricer(90, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.58716171, 0.97606838, 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-40.61799127, -87.77772316,   0.        ],
        [  0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[12.22656241, 26.42227684, 57.1       ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]))

In [70]:
eurBinomPricer(110, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.77723468, 1.        , 0.        ],
        [0.        , 0.44090909, 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-57.08968202, -91.27499672,   0.        ],
        [  0.        , -29.82290419,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[28.40613292, 51.72500328, 90.9       ],
        [ 0.        ,  8.97709581, 19.4       ],
        [ 0.        ,  0.        ,  0.        ]]))

In [71]:
eurBinomPricer(120, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.84892406, 1.        , 0.        ],
        [0.        , 0.62083333, 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-65.0523151 , -91.27499672,   0.        ],
        [  0.        , -45.81044046,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[ 36.81857238,  64.72500328, 107.8       ],
        [  0.        ,  13.78955954,  29.8       ],
        [  0.        ,   0.        ,   0.        ]]))

In [72]:
eurBinomPricer(130, 95, 0.08, 0, 1, 2,1.3, 0.8, callPayoff, verbose=False)

(array([[0.90958431, 1.        , 0.        ],
        [0.        , 0.77307692, 0.        ],
        [0.        , 0.        , 0.        ]]),
 array([[-73.01494817, -91.27499672,   0.        ],
        [  0.        , -61.79797673,   0.        ],
        [  0.        ,   0.        ,   0.        ]]),
 array([[ 45.23101184,  77.72500328, 124.7       ],
        [  0.        ,  18.60202327,  40.2       ],
        [  0.        ,   0.        ,   0.        ]]))

The delta is getting closer and closer to the value of 1.  It rises to one at a quicker rate and begins to slow as the spot price grows greater than the strike price.  While the stock price increases the option is much more likely to be excercised, which means that it will be held to maturity.    

PROBLEM 5

In [75]:
euroBinomPricerRecursive(100, 95, 0.08, 0.3, 0, 1, 3, callPayoff, verbose=False)

18.28255220737056

The American call option premium is 18.2826.  The american call option premium is the same as the european call option premium are the same because there are no dividends.  

In [76]:
print("Premium for the European put is ",euroBinomPricerRecursive(100, 95, 0.08, 0.3, 0, 1, 3, putPayoff, verbose=False))

Premium for the European put is  5.97860511410097


In [78]:
#Put-call parity
print(5.978605-18.2826)
print((95/((1+0.08)**1))-100)

-12.303994999999999
-12.037037037037038


The values differ by about 30 percentage points, therefore Put-call parity appears to hold.

PROBLEM 6

In [81]:
print("u = ", np.exp((0.08 - 0) * (0.5/3) + 0.3 * np.sqrt(0.5/3)))
print("d = ", np.exp((0.08 - 0) * (0.5/3) - 0.3 * np.sqrt(0.5/3)))

u =  1.1454617381452392
d =  0.8966038495199921


In [84]:
print("Premium for the European call is ", euroBinomPricerRecursive(40, 40, 0.08, 0.3, 0, 0.5, 3, callPayoff, verbose=False))

Premium for the European call is  4.377429513517246


In [85]:
print("Premium for the European put is ", euroBinomPricerRecursive(40, 40, 0.08, 0.3, 0, 0.5, 3, putPayoff, verbose=False))

Premium for the European put is  2.8090070796101805


In [86]:
print("Premium for the American put is ", americanBinomPricer(40, 40, 0.08, 0.3, 0, 0.5, 3, putPayoff, verbose=False))

Premium for the American put is  [40.         40.         40.          0.86799019]


The premium for the American call is 4.3774 because the dividends are zero.